In [37]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 17: Reservoir Research ---</h2><p>You arrive in the year 18. If it weren't for <span title="Continuity!">the coat</span> you got in 1018, you would be very cold: the North Pole base hasn't even been constructed.</p>
<p>Rather, it hasn't been constructed <em>yet</em>.  The Elves are making a little progress, but there's not a lot of liquid water in this climate, so they're getting very dehydrated.  Maybe there's more underground?</p>
<p>You scan a two-dimensional vertical slice of the ground nearby and discover that it is mostly <em>sand</em> with veins of <em>clay</em>.  The scan only provides data with a granularity of <em>square meters</em>, but it should be good enough to determine how much water is trapped there. In the scan, <code>x</code> represents the distance to the right, and <code>y</code> represents the distance down. There is also a <em>spring of water</em> near the surface at <code>x=500, y=0</code>. The scan identifies <em>which square meters are clay</em> (your puzzle input).</p>
<p>For example, suppose your scan shows the following veins of clay:</p>
<pre><code>x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504
</code></pre>
<p>Rendering clay as <code>#</code>, sand as <code>.</code>, and the water spring as <code>+</code>, and with <code>x</code> increasing to the right and <code>y</code> increasing downward, this becomes:</p>
<pre><code>   44444455555555
   99999900000000
   45678901234567
 0 ......+.......
 1 ............#.
 2 .#..#.......#.
 3 .#..#..#......
 4 .#..#..#......
 5 .#.....#......
 6 .#.....#......
 7 .#######......
 8 ..............
 9 ..............
10 ....#.....#...
11 ....#.....#...
12 ....#.....#...
13 ....#######...
</code></pre>
<p>The spring of water will produce water <em>forever</em>. Water can move through sand, but is blocked by clay. Water <em>always moves down</em> when possible, and spreads to the left and right otherwise, filling space that has clay on both sides and falling out otherwise.</p>
<p>For example, if five squares of water are created, they will flow downward until they reach the clay and settle there. Water that has come to rest is shown here as <code>~</code>, while sand through which water has passed (but which is now dry again) is shown as <code>|</code>:</p>
<pre><code>......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#.|#......
.#....|#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
</code></pre>
<p>Two squares of water can't occupy the same location.  If another five squares of water are created, they will settle on the first five, filling the clay reservoir a little more:</p>
<pre><code>......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#.|#......
.#~~~~~#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
</code></pre>
<p>Water pressure does not apply in this scenario. If another four squares of water are created, they will stay on the right side of the barrier, and no water will reach the left side:</p>
<pre><code>......+.......
......|.....#.
.#..#.|.....#.
.#..#~~#......
.#..#~~#......
.#~~~~~#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
</code></pre>
<p>At this point, the top reservoir overflows. While water can reach the tiles above the surface of the water, it cannot settle there, and so the next five squares of water settle like this:</p>
<pre><code>......+.......
......|.....#.
.#..#<em>|</em>|||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
........|.....
....#...|.#...
....#...|.#...
....#~~~~~#...
....#######...
</code></pre>
<p>Note especially the leftmost <code>|</code>: the new squares of water can reach this tile, but cannot stop there.  Instead, eventually, they all fall to the right and settle in the reservoir below.</p>
<p>After 10 more squares of water, the bottom reservoir is also full:</p>
<pre><code>......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
........|.....
....#~~~~~#...
....#~~~~~#...
....#~~~~~#...
....#######...
</code></pre>
<p>Finally, while there is nowhere left for the water to settle, it can reach a few more tiles before overflowing beyond the bottom of the scanned data:</p>
<pre><code>......+.......    (line not counted: above minimum y value)
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..
...|.......|..    (line not counted: below maximum y value)
...|.......|..    (line not counted: below maximum y value)
...|.......|..    (line not counted: below maximum y value)
</code></pre>
<p>How many tiles can be reached by the water?  <em>To prevent counting forever</em>, ignore tiles with a <code>y</code> coordinate smaller than the smallest <code>y</code> coordinate in your scan data or larger than the largest one. Any <code>x</code> coordinate is valid. In this example, the lowest <code>y</code> coordinate given is <code>1</code>, and the highest is <code>13</code>, causing the water spring (in row <code>0</code>) and the water falling off the bottom of the render (in rows <code>14</code> through infinity) to be ignored.</p>
<p>So, in the example above, counting both water at rest (<code>~</code>) and other sand tiles the water can hypothetically reach (<code>|</code>), the total number of tiles the water can reach is <code><em>57</em></code>.</p>
<p><em>How many tiles can the water reach</em> within the range of <code>y</code> values in your scan?</p>
</article>


In [38]:
from collections import deque
from more_itertools import first


example = """
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504
"""


class VeinsOfClay:
    def __init__(self, s: str) -> None:
        rects, min_x, max_x, min_y, max_y = self._parse(s)

        self.map = [
            ["#" if (x, y) in rects else "." for x in range(min_x - 1, max_x + 2)]
            for y in range(max_y + 2)
        ]

        self.map[0][500 - min_x + 1] = "+"
        self.spring = 500 - min_x + 1, 0
        self.min_y = min_y

        self._open_spring()

    def _parse(self, s):
        rects = set()
        min_x, max_x = 500, 500
        min_y, max_y = inf, -inf

        for line in s.strip().splitlines():
            for sl in line.split(", "):
                pos = [int(i) for i in re.findall(r"\d+", sl)]
                if len(pos) == 1:
                    fr = first(pos)
                    to = fr + 1
                else:
                    fr, to = pos
                    to += 1
                if sl.startswith("x"):
                    x_range = fr, to
                    max_x = max(to - 1, max_x)
                    min_x = min(fr, min_x)
                else:
                    y_range = fr, to
                    max_y = max(to - 1, max_y)
                    min_y = min(fr, min_y)

            for x, y in product(range(*x_range), range(*y_range)):
                rects.add((x, y))
        return rects, min_x, max_x, min_y, max_y

    def _open_spring(self):
        x, y = self.spring
        queue = deque([(x, y + 1)])

        while queue:
            x, y = queue.popleft()
            # let water fall
            while 0 <= y + 1 < len(self.map) and self.map[y + 1][x] not in "#|~":
                self.map[y][x] = "|"
                y += 1

            if 0 <= y + 1 < len(self.map) and self.map[y + 1][x] in "#~":
                left_contained, right_contained = True, True
                # find left border
                while left_contained and right_contained:
                    left = 1
                    while (
                        self.map[y][x - left] != "#"
                        and self.map[y + 1][x - left] in "#~"
                    ):
                        left += 1

                    left_contained = self.map[y][x - left] == "#"

                    # find right border
                    right = 1
                    while (
                        self.map[y][x + right] != "#"
                        and self.map[y + 1][x + right] in "#~"
                    ):
                        right += 1

                    right_contained = self.map[y][x + right] == "#"
                    if left_contained and right_contained:
                        for xx in range(x - left + 1, x + right):
                            self.map[y][xx] = "~"

                        y -= 1

                for xx in range(x - left + 1, x + right):
                    self.map[y][xx] = "|"

                if not left_contained:
                    queue.append((x - left, y))

                if not right_contained:
                    queue.append((x + right, y))

            elif 0 <= y + 1 < len(self.map) and self.map[y + 1][x] == "|":
                self.map[y][x] = "|"

    def count_wet_tiles(self) -> int:
        return sum(1 for l in self.map[self.min_y :] for c in l if c in "|~")

    def retained(self) -> int:
        return sum(1 for l in self.map for c in l if c == "~")

    def __repr__(self) -> str:
        return "\n".join("".join(l) for l in self.map)


print(f"Example: {VeinsOfClay(example).count_wet_tiles()} should be 57")

Example: 57 should be 57


In [39]:
test = """
x=480, y=2..14
x=520, y=2..14
y=14, x=480..520
y=6, x=485..500
y=10, x=485..500
x=485, y=6..10
x=500, y=6..10
"""
tv = VeinsOfClay(test)
print(f"{tv.count_wet_tiles()=}, {tv.retained()=}")
tv

tv.count_wet_tiles()=414, tv.retained()=388


.....................+.....................
|||||||||||||||||||||||||||||||||||||||||||
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~################~~~~~~~~~~~~~~~~~~~#|
|#~~~~#..............#~~~~~~~~~~~~~~~~~~~#|
|#~~~~#..............#~~~~~~~~~~~~~~~~~~~#|
|#~~~~#..............#~~~~~~~~~~~~~~~~~~~#|
|#~~~~################~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#|
|#########################################|
...........................................

In [40]:
with open("../input/day17.txt") as f:
    puzzle = f.read()


print(f"Part I: { VeinsOfClay(puzzle).count_wet_tiles()}")  # 42393 is too low

Part I: 42389


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>42389</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After <span title="A very long time ago, I built something similar to this in QBasic. SCREEN 13 forever!">a very long time</span>, the water spring will run dry. How much water will be retained?</p>
<p>In the example above, water that won't eventually drain out is shown as <code>~</code>, a total of <code><em>29</em></code> tiles.</p>
<p><em>How many water tiles are left</em> after the water spring stops producing water and all remaining water not at rest has drained?</p>
</article>

</main>


In [41]:
print(f"Example: { VeinsOfClay(example).retained()} should be 29")

Example: 29 should be 29


In [42]:
print(f"Part II: { VeinsOfClay(puzzle).retained()}")  # 42393 is too low

Part II: 34497


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>34497</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
